In [1]:
import cv2
import pytesseract
import pandas as pd
import re
from PIL import Image
import os

In [2]:
# Replace 'your_receipt_image.jpg' with the path to your image file
#image_path = 'PXL_20240925_014423209~2v1.jpg' # clean image
image_path = 'Walmart_Receipt_20241001.jpg'
#image_path = 'PXL_20240925_014423209~2v1_HighContrast.jpg' # High contrast clean image
#image_path = 'PXL_20240925_014423209~2v1_HighContrastGimp1.png' # High contrast clean image
#image_path = 'PXL_20240923_014803753.MP~2.jpg' # -> image 4
#image_path = 'PXL_20240923_014546368~2.jpg' # -> image 3
#image_path = 'PXL_20240922_025356838~2.jpg' # -> image 2
#image_path = 'PXL_20240922_050714477~2.jpg' # -> image 1
filename , extn = os.path.splitext(image_path)
filepath = "C:\\Users\\sunil\\Desktop\\"
output_file_name = filename + '.txt'
output_file = filepath + output_file_name
print(output_file)
img = cv2.imread(image_path)

C:\Users\sunil\Desktop\Walmart_Receipt_20241001.txt


In [3]:
# Preprocessing: Convert to grayscale, blur, and threshold
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
gray = cv2.GaussianBlur(gray, (3, 3), 0)
_, binary_img = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

# Dilation to enhance the text
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))
dilated_img = cv2.dilate(binary_img, kernel, iterations=1)

# Perform OCR
custom_config = r'--oem 3 --psm 6'
extracted_text = pytesseract.image_to_string(dilated_img, config=custom_config)

In [4]:
print(extracted_text)

Save money. Live better.
WAL*MART
TBD Mgr. TBD
5200 WINDWARD PKWY
ALPHARETTA, GA
ST# 2941 OP# 9016 TE# 16 TR# 4991
TAB INDX CR 062603007216 2.42
ULTRATAB 007276274730 4.62
JPKT CHECK 695803706310 4.96
PG lz PRT S 695803709659 4.44
SUBTOTAL 14.34
TAX 17.75 1.11
TOTAL 15.45
MASTERCARD DEBIT TEND 15.45
MASTERCARD wae wee wee LOOT
CHANGE DUE 0.00
TCH 4441 z2e98 2144 1311 5248
10/01/24 09:10:43



In [5]:
# Define regular expressions to capture the two formats of Walmart Headers

# Format 1: 3 lines Walmart Header
header_format_1 = re.compile(
    r"(You could win a \$1000 GiftCard!\nVisit survey\.walmart\.com#([A-Za-z0-9]+)\nFor more details, see back of receipt\.)", re.MULTILINE
)

# Format 2: 2 lines Walmart Header
header_format_2 = re.compile(
    r"(Give us feedback @ survey\.walmart\.com\nThank you! ID #:(\S+))", re.MULTILINE
)

header_format_3 = re.compile(
    r"(Save money. Live better.)", re.MULTILINE
)

# "Walmart Logo" rule (the line "Walmart ><")
walmart_logo_pattern = re.compile(r"Walmart ><", re.MULTILINE)
walmart_title_pattern = re.compile(r"WAL*MART", re.MULTILINE)

# Check for "Walmart Headers" , "Walmart Logo" and "walmart title". extract the Receipt ID and headers
receipt_id = None
header = None
header_contents = None

# Extract lines before the "Walmart Logo"
header_lines = []
remaining_text = extracted_text.split("\n")  # To store the remaining receipt text

print(remaining_text)

['Save money. Live better.', 'WAL*MART', 'TBD Mgr. TBD', '5200 WINDWARD PKWY', 'ALPHARETTA, GA', 'ST# 2941 OP# 9016 TE# 16 TR# 4991', 'TAB INDX CR 062603007216 2.42', 'ULTRATAB 007276274730 4.62', 'JPKT CHECK 695803706310 4.96', 'PG lz PRT S 695803709659 4.44', 'SUBTOTAL 14.34', 'TAX 17.75 1.11', 'TOTAL 15.45', 'MASTERCARD DEBIT TEND 15.45', 'MASTERCARD wae wee wee LOOT', 'CHANGE DUE 0.00', 'TCH 4441 z2e98 2144 1311 5248', '10/01/24 09:10:43', '']


In [6]:
print(type(extracted_text))

<class 'str'>


In [7]:
for line in extracted_text.split("\n"):
    if "WAL*MART" in line:
        break  # Stop when we reach the "Walmart Logo"
    header_lines.append(line)

print(header_lines)


['Save money. Live better.']


In [8]:
# Remove the detected "Walmart Headers" from the remaining text
header_block = "\n".join(header_lines)
remaining_text = "\n".join(remaining_text[len(header_lines)+1:])  # Skip Walmart Logo

# Check which header format matches and extract the Receipt ID
if header_format_1.search(header_block):
    header = "Format 1 (3 lines)"
    match = header_format_1.search(header_block)
    header_contents = match.group(1)
    receipt_id = match.group(2)
elif header_format_2.search(header_block):
    header = "Format 2 (2 lines)"
    match = header_format_2.search(header_block)
    header_contents = match.group(1)
    receipt_id = match.group(2)
elif header_format_3.search(header_block):
    header = "Format 3 (1 line)"
    match = header_format_3.search(header_block)
    header_contents = match.group(1)
    #receipt_id = match.group(2)

# Print the Walmart Headers, Receipt ID, and Walmart Logo
if header:
    print(f"Walmart Headers Detected: {header}")
    print(f"Receipt ID: {receipt_id}")
    print("\nWalmart Header Content:")
    print(header_contents)
else:
    print("No Walmart Headers detected.")

# Check for "Walmart Logo" and print it
logo_match = walmart_logo_pattern.search(extracted_text)
if logo_match:
    print("\nWalmart Logo detected:")
    print("Walmart ><")

title_match = walmart_title_pattern.search(extracted_text)
if title_match:
    print("\nWalmart Title detected:")
    print("Wal*mart")

# After detection, we remove the header and logo from further use
print("\nRemaining Text after removing Walmart Headers and Walmart Logo:")
print(remaining_text)

Walmart Headers Detected: Format 3 (1 line)
Receipt ID: None

Walmart Header Content:
Save money. Live better.

Remaining Text after removing Walmart Headers and Walmart Logo:
TBD Mgr. TBD
5200 WINDWARD PKWY
ALPHARETTA, GA
ST# 2941 OP# 9016 TE# 16 TR# 4991
TAB INDX CR 062603007216 2.42
ULTRATAB 007276274730 4.62
JPKT CHECK 695803706310 4.96
PG lz PRT S 695803709659 4.44
SUBTOTAL 14.34
TAX 17.75 1.11
TOTAL 15.45
MASTERCARD DEBIT TEND 15.45
MASTERCARD wae wee wee LOOT
CHANGE DUE 0.00
TCH 4441 z2e98 2144 1311 5248
10/01/24 09:10:43



In [9]:
# Step 3: Detect Store Details section (5 lines after the Walmart Logo)
store_details_lines = remaining_text.split("\n")[:4]  # Capture 4 lines after the logo
print(store_details_lines)

['TBD Mgr. TBD', '5200 WINDWARD PKWY', 'ALPHARETTA, GA', 'ST# 2941 OP# 9016 TE# 16 TR# 4991']


In [10]:


if len(store_details_lines) == 4:
    print("\nStore Details Section Detected:")

    
    # Line 1: Store type and name (e.g., "WM Supercenter")
    store_type_match = re.search(r"(Supercenter|Discount Store|Neighborhood Market|Fuel Station|Grocery Pickup|Deli)", store_details_lines[0])
    store_type = store_type_match.group(0) if store_type_match else "Unknown Store Type"
    print(f"Store Type: {store_type}")
    
    
    # Line 2: Store phone number and manager name
    phone_manager_match = re.search(r"(\d{3}-\d{3}-\d{4})\s+Mgr\.\s+([A-Za-z]+)", store_details_lines[0])
    if phone_manager_match:
        phone_number = phone_manager_match.group(1)
        manager_name = phone_manager_match.group(2)
        print(f"Phone Number: {phone_number}")
        print(f"Manager: {manager_name}")
    
    # Line 3: Street number and name
    print(f"Street Address: {store_details_lines[1]}")
    
    # Line 4: City, State, and ZIP code
    print(f"City, State, ZIP: {store_details_lines[2]}")
    
    # Line 5: ST#, OP#, TE#, TR# extraction
    line_5_match = re.search(r"ST#\s*(\d+)\s+OP#\s*(\d+)\s+TE#\s*(\d+)\s+TR#\s*(\d+)", store_details_lines[3])
    if line_5_match:
        store_number = line_5_match.group(1)
        operator_number = line_5_match.group(2)
        terminal_number = line_5_match.group(3)
        transaction_number = line_5_match.group(4)
        print(f"Store Number (ST#): {store_number}")
        print(f"Operator Number (OP#): {operator_number}")
        print(f"Terminal Number (TE#): {terminal_number}")
        print(f"Transaction Number (TR#): {transaction_number}")
    
    # Remove the "Store Details" section from the remaining text
    remaining_text = "\n".join(remaining_text.split("\n")[4:])
else:
    print("Store Details section not detected properly.")



Store Details Section Detected:
Store Type: Unknown Store Type
Street Address: 5200 WINDWARD PKWY
City, State, ZIP: ALPHARETTA, GA
Store Number (ST#): 2941
Operator Number (OP#): 9016
Terminal Number (TE#): 16
Transaction Number (TR#): 4991


In [11]:
# Step 4: Detect "# ITEMS SOLD" section and find number of items sold
items_sold_pattern = re.compile(r"# ITEMS SOLD\s+(\d+)", re.MULTILINE)
items_sold_match = items_sold_pattern.search(remaining_text)

if items_sold_match:
    items_sold = items_sold_match.group(1)
    print(f"\nCount of Items Purchased Detected: {items_sold} items sold.")
    
    # Remove this section from the remaining text
    remaining_text = items_sold_pattern.sub("", remaining_text)
else:
    print("\nCount of Items Purchased section not detected.")


Count of Items Purchased section not detected.


In [12]:
# Step 5: Detect "Transaction #" section, allowing for common OCR errors (like TCH instead of TC#)
transaction_pattern = re.compile(r"T[CCH#]+\s+(\d{4}\s+\d{4}\s+\d{4}\s+\d{4}\s+\d{4})", re.MULTILINE)
transaction_match = transaction_pattern.search(remaining_text)

if transaction_match:
    transaction_number = transaction_match.group(1)
    print(f"\nTransaction # Detected: TC# {transaction_number}")
    
    # Remove this section from the remaining text
    remaining_text = transaction_pattern.sub("", remaining_text)
else:
    print("\nTransaction # section not detected.")

# Print remaining text after removing "Store Details", "Count of Items Purchased", and "Transaction #"
print("\nRemaining Text after removing Transaction #:")
print(remaining_text)


Transaction # section not detected.

Remaining Text after removing Transaction #:
TAB INDX CR 062603007216 2.42
ULTRATAB 007276274730 4.62
JPKT CHECK 695803706310 4.96
PG lz PRT S 695803709659 4.44
SUBTOTAL 14.34
TAX 17.75 1.11
TOTAL 15.45
MASTERCARD DEBIT TEND 15.45
MASTERCARD wae wee wee LOOT
CHANGE DUE 0.00
TCH 4441 z2e98 2144 1311 5248
10/01/24 09:10:43



In [13]:

# Extract text before "SUBTOTAL"
extracted_text2 = remaining_text.split("SUBTOTAL")[0].strip()

# Output the result
print(extracted_text2)

TAB INDX CR 062603007216 2.42
ULTRATAB 007276274730 4.62
JPKT CHECK 695803706310 4.96
PG lz PRT S 695803709659 4.44


In [14]:
# Define regex patterns
single_line_pattern = re.compile(r'^.*\d{10,12}.*\d+\.\d{2}.*$')
#weight_item_pattern = re.compile(r'^\d+\.\d{3}\s*lb\.\s*@\s*\d+\s*lb\.|oz.*\d+\.\d{2}.*$')
weight_item_pattern = re.compile(r'^\d+\.\d{3}\s*[lI]b\.\s*@\s*\d+\s*[lI]b\.|oz.*\d+\.\d{2}.*$')


# Split the text into lines
lines = extracted_text2.strip().split('\n')

# Initialize lists to store categorized items
one_line_items = []
two_line_items = []

# Loop through the lines and categorize items
i = 0
while i < len(lines):
    line = lines[i].strip()
    
    # Check if the line matches a single-line item
    if single_line_pattern.match(line):
        one_line_items.append(line)
    
    # Check if it's the start of a two-line item (non-priced weighted items)
    elif i + 1 < len(lines) and weight_item_pattern.match(lines[i + 1]):
        two_line_items.append(f"{line}\n{lines[i + 1]}")
        i += 1  # Skip the next line since it's part of the two-line item
    
    i += 1

# Output the results
print("Items with complete information in one line:")
for item in one_line_items:
    print(item)

print("\nItems with information spanning two lines:")
for item in two_line_items:
    print(item)

Items with complete information in one line:
TAB INDX CR 062603007216 2.42
ULTRATAB 007276274730 4.62
JPKT CHECK 695803706310 4.96
PG lz PRT S 695803709659 4.44

Items with information spanning two lines:


In [15]:
# The new regex to extract data from one_line_items
one_line_regex = re.compile(r"([A-Za-z\s]+[a-zA-Z0-9\s]+)\s+(\d{10,12}.)\s*([FR]?)\s+(\d+\.\d{2})\s*([xXR]?)")

print("Extracted Information from one-line items:")
for item in one_line_items:
    match = one_line_regex.match(item)
    if match:
        item_name = match.group(1)
        item_code = match.group(2)
        optional_FR = match.group(3)
        price = match.group(4)
        optional_XR = match.group(5)
        
        # Display the extracted information
        print(f"Item: {item_name}, Code: {item_code}, F/R: {optional_FR}, Price: {price}, X/R: {optional_XR}")

Extracted Information from one-line items:
Item: TAB INDX CR, Code: 062603007216, F/R: , Price: 2.42, X/R: 
Item: ULTRATAB, Code: 007276274730, F/R: , Price: 4.62, X/R: 
Item: JPKT CHECK, Code: 695803706310, F/R: , Price: 4.96, X/R: 
Item: PG lz PRT S, Code: 695803709659, F/R: , Price: 4.44, X/R: 


In [16]:
# New regex for two-line items
combined_regex = re.compile(
    r"([A-Za-z\s]+[A-Za-z])\s+(\d{10,12})\s*([FR]?)\s+(\d+\.\d{3})\s*(lb|Ib)\s*@\s*(\d+)\s*(lb|Ib)\s*/\s*(\d+\.\d{2})\s*(\d+\.\d{2})\s*([XR]?)"
)
two_line_items_list = []
for item in two_line_items:
    
    item_combined = item.replace('\n', ' ')
    two_line_items_list.append(item_combined)
    print(f"Processing item: {item_combined}")  # To inspect the exact format after combining lines

In [17]:
for item in two_line_items_list:
    print(item)

In [18]:

# print the text for weighted items
for i in two_line_items_list:
    print(i)
# Debugging: Print the text we're analyzing
print("\nText for Analysis:")
for i, line in enumerate(two_line_items_list):
    print(f"Line {i+1}: {line}")

# Step 1: New simplified regex pattern to capture item name, item code, and price details
# Adjusting the pattern to handle price per unit and total price correctly
weighted_item_pattern = re.compile(
    r"([A-Za-z\s]+)\s+(\d{10,12})\s*([FR]?)\s+(\d+\.\d{3}\s+(lb|Ib)\.|oz\.)\s+@\s+1\s+(lb|Ib)\.\s+\/\s*(\d+\.\d{2})\s+(\d+\.\d{2})\s*([XR]?)"
)

# Find all weighted items from the list of lines
weighted_items = []
for line in two_line_items_list:
    match = weighted_item_pattern.findall(line)
    if match:
        weighted_items.extend(match)

# Step 2: Display the weighted item list
print("\nWeighted Item List:")
if not weighted_items:
    print("No weighted items found.")
else:
    for item in weighted_items:
        item_name = item[0].strip()
        item_code = item[1]
        optional_code = item[2]
        units_purchased_vs_price_per_unit = f"{item[3]} @ 1 lb. / {item[4]}"  # Combine weight and price per unit
        total_price = item[5]
        price_type = item[6]
        print(f"Item: {item_name}, Code: {item_code}, Units Purchased vs Price per Unit: {units_purchased_vs_price_per_unit}, Total Price: {total_price}, Price Type: {price_type}")

# Step 3: Additional debugging if needed
if not weighted_items:
    print("\nNo matches were found. Let's debug:")
    for line in two_line_items_list:
        print(f"\nAnalyzing line: {line}")
        for match in re.finditer(r"\d{10,12}", line):
            print(f"Item Code match at position {match.start()}: {match.group(0)}")
        for match in re.finditer(r"\d+\.\d{3}\s+(lb\.|oz\.)", line):
            print(f"Weight match at position {match.start()}: {match.group(0)}")
        for match in re.finditer(r"@\s+1\s+lb\.\s+\/\s*\d+\.\d{2}", line):
            print(f"Units Purchased vs Price per Unit match at position {match.start()}: {match.group(0)}")
        for match in re.finditer(r"\d+\.\d{2}", line):
            print(f"Total Price match at position {match.start()}: {match.group(0)}")


Text for Analysis:

Weighted Item List:
No weighted items found.

No matches were found. Let's debug:
